<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [8]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [9]:
import folium
import pandas as pd

In [17]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [22]:
## Task 1: Mark all launch sites on a map
# Calculate the mean latitude and longitude to center the map
nasa_coordinate = [launch_sites_df['Lat'].mean(), launch_sites_df['Long'].mean()]

# Initialize the map
# We start with a zoom_start of 5 to see a broader area.
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

print("\nAdding markers and circles for each launch site...")

# Add a marker and a circle for each launch site
for index, site in launch_sites_df.iterrows():
    coordinate = [site['Lat'], site['Long']]
    site_name = site['Launch Site']

    # Add a Circle object (often used to visualize a region around the site)
    folium.Circle(
        location=coordinate,
        radius=1000, # Radius in meters (e.g., 1 km)
        color='#000000', # Black border
        fill=True,
        fill_color='#d35400', # Orange fill
        fill_opacity=0.7
    ).add_child(folium.Popup(site_name)).add_to(site_map) # Popup shows site name

    # Add a Marker with a custom DivIcon for the site name label
    folium.Marker(
        location=coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12pt; color:#d35400;"><b>{site_name}</b></div>'
        )
    ).add_to(site_map)

print("\nMap with all launch sites (circles and labels) created.")

# Display the map
site_map



Adding markers and circles for each launch site...

Map with all launch sites (circles and labels) created.


In [21]:
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [19]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [20]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [23]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)
site_map

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [24]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`



In [26]:

# --- Prepare launch_sites_df ---
# Extract unique launch sites and their coordinates.
# Assuming 'Latitude' and 'Longitude' are the correct column names (initial capital).
# If you get a KeyError here, refer to the "IMPORTANT" note above.
launch_sites_df = df[['LaunchSite', 'Latitude', 'Longitude']].drop_duplicates().reset_index(drop=True)

print("\nUnique Launch Sites and their Coordinates:")
print(launch_sites_df)

# --- TASK 1: Mark all launch sites on a map ---

# Calculate the mean latitude and longitude to center the map
nasa_coordinate = [launch_sites_df['Latitude'].mean(), launch_sites_df['Longitude'].mean()]

# Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

print("\nAdding circles and markers for each launch site...")

# Add a marker and a circle for each launch site
for index, site in launch_sites_df.iterrows():
    coordinate = [site['Latitude'], site['Longitude']]
    site_name = site['LaunchSite']

    # Add a Circle object (often used to visualize a region around the site)
    folium.Circle(
        location=coordinate,
        radius=1000, # Radius in meters (e.g., 1 km)
        color='#000000', # Black border
        fill=True,
        fill_color='#d35400', # Orange fill
        fill_opacity=0.7
    ).add_child(folium.Popup(site_name)).add_to(site_map) # Popup shows site name

    # Add a Marker with a custom DivIcon for the site name label
    folium.Marker(
        location=coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12pt; color:#d35400;"><b>{site_name}</b></div>'
        )
    ).add_to(site_map)

print("\nMap with all launch sites (circles and labels) created.")


# Display the map
site_map

<class 'NameError'>: name 'df' is not defined

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


* No, not all launch sites are in very close proximity to the Equator line
* Yes, almost all orbital launch sites are located in very close proximity to the coast, or at least adjacent to a large, unpopulated body of water (like a large desert or ocean)

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [27]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [28]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [52]:
import numpy as np
print("\nCreating 'marker_color' column...")
df['marker_color'] = np.where(df['Class'] == 1, 'green', 'red')

print("'marker_color' column created successfully.")
print("\nDataFrame head with new 'marker_color' column:")
print(df.head())
print("\nValue counts for 'marker_color':")
print(df['marker_color'].value_counts())
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red


Creating 'marker_color' column...
'marker_color' column created successfully.

DataFrame head with new 'marker_color' column:
   FlightNumber        Date BoosterVersion  PayloadMass Orbit    LaunchSite  \
0             1  2010-06-04       Falcon 9  6104.959412   LEO  CCAFS SLC 40   
1             2  2012-05-22       Falcon 9   525.000000   LEO  CCAFS SLC 40   
2             3  2013-03-01       Falcon 9   677.000000   ISS  CCAFS SLC 40   
3             4  2013-09-29       Falcon 9   500.000000    PO   VAFB SLC 4E   
4             5  2013-12-03       Falcon 9  3170.000000   GTO  CCAFS SLC 40   

       Outcome  Flights  GridFins  Reused   Legs LandingPad  Block  \
0    None None        1     False   False  False        NaN    1.0   
1    None None        1     False   False  False        NaN    1.0   
2    None None        1     False   False  False        NaN    1.0   
3  False Ocean        1     False   False  False        NaN    1.0   
4    None None        1     False   False  False

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [54]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [2]:
import piplite
await piplite.install(['geopy'])
from geopy.distance import geodesic

launch_site_coords = (28.573255, -80.646897) # Latitude, Longitude for LC-39A

print(f"Reference Launch Site: Cape Canaveral LC-39A at {launch_site_coords}")

# Define coordinates for various proximity features (examples for Florida)
# You might need to adjust these based on your specific lab's points of interest
proximity_points = {
    "Closest City (Titusville)": (28.6120, -80.8086),
    "Closest Coastline (Playalinda Beach)": (28.6186, -80.6057),
    "Closest Railway (Near Cocoa-Rockledge Amtrak)": (28.3789, -80.7891),
    "Closest Highway (I-95 & SR-528 Intersection)": (28.4039, -80.8249),
}

print("\nCalculating distances to proximities:")

# Calculate and print distances
for name, coords in proximity_points.items():
    distance = geodesic(launch_site_coords, coords).km
    print(f"- Distance to {name}: {distance:.2f} km")


Reference Launch Site: Cape Canaveral LC-39A at (28.573255, -80.646897)

Calculating distances to proximities:
- Distance to Closest City (Titusville): 16.39 km
- Distance to Closest Coastline (Playalinda Beach): 6.44 km
- Distance to Closest Railway (Near Cocoa-Rockledge Amtrak): 25.65 km
- Distance to Closest Highway (I-95 & SR-528 Intersection): 25.61 km


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [57]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [58]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [33]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
nasa_coordinate = [launch_sites_df['Lat'].mean(), launch_sites_df['Long'].mean()]

# --- Initialize the map with MousePosition ---
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add MousePosition to the map
formatter = "function(num) {return L.Util.formatNum(num, 5) + 'º';};"
MousePosition(
    position='topright',
    separator=' | ',
    empty_string='NaN',
    lng_first=True,
    num_digits=20,
    prefix='Coordinates:',
    lat_formatter=formatter,
    lng_formatter=formatter
).add_to(site_map)

print("Map initialized with MousePosition. Please run this cell and interact with the map below.")
print("Move your mouse over the coastline and note down the latitude and longitude.")

# Display the map. After running, you'll see a map below.
# Move your mouse cursor over the coastline on the map and note the coordinates.
site_map

Map initialized with MousePosition. Please run this cell and interact with the map below.
Move your mouse over the coastline and note down the latitude and longitude.


In [4]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )
coastline_point_coords = (28.28201, -80.69321)

# Define the coordinates for the launch site you want to measure from
# Using Cape Canaveral LC-39A as an example, as in Task 3
launch_site_coords = (28.573255, -80.646897) # Example: Cape Canaveral LC-39A

# Calculate the distance
distance_to_coastline = geodesic(launch_site_coords, coastline_point_coords).km

print(f"\nLaunch Site Coordinates: {launch_site_coords}")
print(f"Coastline Point Coordinates: {coastline_point_coords}")
print(f"The distance between the launch site and the marked coastline point is: {distance_to_coastline:.2f} km")


Launch Site Coordinates: (28.573255, -80.646897)
Coastline Point Coordinates: (28.28201, -80.69321)
The distance between the launch site and the marked coastline point is: 32.59 km


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [31]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
nasa_coordinate = [launch_sites_df['Lat'].mean(), launch_sites_df['Long'].mean()]

# --- Initialize the map with MousePosition ---
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
launch_site_coords = [28.573255, -80.646897] # Latitude, Longitude for LC-39A

# Using the example coastline coordinates you provided:
coastline_point_coords = [28.56367, -80.57163] # Latitude, Longitude for the closest coastline point

# --- Create a `folium.PolyLine` object ---
# The PolyLine needs a list of points.
# The order matters for drawing the line from start to end.
points = [launch_site_coords, coastline_point_coords]

# Create the PolyLine
lines = folium.PolyLine(
    locations=points,
    weight=5, # Line thickness
    color='blue', # Line color
    tooltip="Distance to Coastline" # Tooltip when hovering over the line
)

# Add the PolyLine to the map
site_map.add_child(lines)
site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
